# Tools - Function Calling 

The **Function Calling Tool** in the OpenAI Assistants API allows you to describe custom functions to the model and have it intelligently determine when and how to call them based on user queries. This enables the creation of assistants that can interact with external systems, fetch data, and perform complex tasks, making them more dynamic and useful.

By the end of this lesson, you will:

1.  Understand the core concepts of function calling.
2.  Learn how to define and register functions with the Assistants API.
3.  Implement two real-world examples:
    *   A **Cooking Assistant** that suggests recipes and provides nutritional information.
    *   A **Package Tracking Assistant** that retrieves package status and delivery dates.
4.  Gain insights into handling function calls and managing the assistant's lifecycle.

## Core Concepts of Function Calling

To effectively use the Function Calling Tool, you need to grasp these key concepts:

1.  **Function Definition**:
    *   Each function must be clearly defined with a name, description, and parameters.
    *   The parameters are described using a JSON schema, specifying their types, descriptions, and whether they are required.
2.  **Registering Functions**:
    *   Functions are registered with the assistant during its creation using the `tools` parameter.
    *   Each function is defined as a tool of type `function`.
3.  **Run Lifecycle**:
    *   An assistant's interaction with a user is managed through a `run` object.
    *   Runs go through different states: `queued`, `in_progress`, `requires_action`, `completed`, `cancelled`, `failed`, `expired`.
    *   When a run enters the `requires_action` state, it means the assistant has determined that one or more functions need to be called.
4.  **Handling Function Calls**:
    *   Your application needs to monitor the run's state.
    *   When a function call is required, the assistant provides the function name and arguments.
    *   You execute the function (or a mock/simulation in our examples) and submit the results back to the assistant.
5.  **Submitting Tool Outputs**:
    *   All tool outputs for a given `requires_action` state must be submitted together.
    *   The `submit_tool_outputs` method is used to send the results back to the assistant.
6.  **Expiration**:
    *   Function calls have a 10-minute expiration window. If outputs are not submitted within this time, the run will expire.
7.  **Streaming**:
    *   The API supports streaming responses, allowing for real-time interactions.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

## Example 1: Building a Smart Cooking Assistant

Let's start by building a cooking assistant that can suggest recipes based on available ingredients and provide nutritional information.

### Step 1: Define the Functions

We'll define two functions:

*   `get_recipe_suggestion`: Suggests recipes based on ingredients and optional cuisine preference.
*   `get_nutritional_info`: Provides nutritional information for a given recipe.

In [ ]:
assistant = client.beta.assistants.create(
    name="Cooking Assistant",
    instructions="You are a helpful cooking assistant. Use the provided functions to suggest recipes and provide nutritional information.",
    model="gpt-4o",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_recipe_suggestion",
                "description": "Get recipe suggestions based on available ingredients",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ingredients": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of available ingredients",
                        },
                        "cuisine_type": {
                            "type": "string",
                            "description": "Preferred cuisine type (optional)",
                            "enum": ["Italian", "Mexican", "Asian", "American", "Any"],
                        },
                    },
                    "required": ["ingredients"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_nutritional_info",
                "description": "Get nutritional information for a recipe",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "recipe_name": {
                            "type": "string",
                            "description": "Name of the recipe",
                        },
                    },
                    "required": ["recipe_name"],
                },
            },
        },
    ],
)

### Step 2: Create a Thread and Add a User Message

Now, let's create a conversation thread and add a user's message to start the interaction.

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I have chicken, rice, and bell peppers. What can I make for dinner?",
)

### Step 3: Handle Function Calls with an Event Handler

We'll use an event handler to process function calls and provide results back to the assistant.

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler

class CookingAssistantHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        tool_outputs = []
        
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_recipe_suggestion":
                # Simulate recipe suggestion function
                recipe = {
                    "name": "Chicken Stir-Fry",
                    "cuisine": "Asian",
                    "cooking_time": "30 minutes",
                }
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": str(recipe),
                })
            elif tool.function.name == "get_nutritional_info":
                # Simulate nutritional information function
                nutrition = {
                    "calories": 400,
                    "protein": "35g",
                    "carbs": "45g",
                    "fat": "12g",
                }
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": str(nutrition),
                })
        
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=CookingAssistantHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()

### Step 4: Run the Assistant and Process the Response

Finally, we'll create a run and use our event handler to manage the interaction.

In [ ]:
# Create and stream the run
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=CookingAssistantHandler()
) as stream:
    stream.until_done()

When you run this code, you'll see the assistant suggest "Chicken Stir-Fry" based on the provided ingredients. If you ask for nutritional information about the suggested recipe, it will provide the simulated data.

## Example 2: Building a Package Tracking Assistant

Now, let's build a Package Tracking Assistant that can retrieve the status and expected delivery date of a package.

### Step 1: Define the Functions

We'll define two functions:

*   `get_package_status`: Retrieves the status of a package based on its tracking number.
*   `get_expected_delivery`: Provides the estimated delivery date for a package.

In [1]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    instructions="You are a package tracking assistant. Use the provided functions to retrieve package status and expected delivery dates.",
    model="gpt-4o",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_package_status",
                "description": "Retrieve the status of a package by tracking number.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "tracking_number": {
                            "type": "string",
                            "description": "The package tracking number.",
                        },
                    },
                    "required": ["tracking_number"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_expected_delivery",
                "description": "Get the expected delivery date for a package.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "tracking_number": {
                            "type": "string",
                            "description": "The package tracking number.",
                        },
                    },
                    "required": ["tracking_number"],
                },
            },
        },
    ],
)

### Step 2: User Query and Function Invocation

Let's create a thread and add a user query:

In [2]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you track package 1234567890?",
)

### Step 3: Create a Run

We will now create a run to process the user's message.

In [3]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

### Step 4: Mock External API Calls and Submit Results

We'll simulate the external API calls and submit the results back to the assistant.

In [4]:
import time

# Wait for the run to require action (or poll for the status)
while run.status != "requires_action":
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Check if the run requires action
if run.status == "requires_action":
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    tool_outputs = []

    for tool_call in tool_calls:
        if tool_call.function.name == "get_package_status":
            # Simulate get_package_status function
            # output = get_package_status(tool_call.function.arguments.tracking_number)
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "output": '{"status": "In Transit"}',
            })
        elif tool_call.function.name == "get_expected_delivery":
            # Simulate get_expected_delivery function
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "output": '{"expected_delivery": "2025-01-15"}',
            })

    # Submit tool outputs
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Step 5: Retrieve and Display the Final Message

Finally, we'll retrieve and display the assistant's final response.

In [5]:
# Wait for the run to complete
while run.status != "completed":
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Retrieve the messages
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Find and print the last assistant message
for msg in reversed(messages.data):
    if msg.role == "assistant":
        print(msg.content[0].text.value)
        break

The package with tracking number 1234567890 is currently "In Transit" and is expected to be delivered on January 15, 2025.


When you run this code, you should see an output similar to:

In [ ]:
# Your package (1234567890) is currently In Transit. It is expected to be delivered by 2025-01-15.

## Conclusion

The Function Calling Tool in the OpenAI Assistants API provides a powerful way to extend the capabilities of your assistants. By integrating external functions, you can create intelligent agents that can perform a wide range of tasks, from providing cooking recommendations to tracking packages and beyond.

In this lesson, we've covered the fundamental concepts of function calling, walked through two practical examples, and provided insights into handling function calls and managing the assistant's lifecycle. With this knowledge, you're well-equipped to start building your own sophisticated assistants using the OpenAI Assistants API.

## Further Exploration

To deepen your understanding and explore more advanced use cases, consider the following:

1.  **Error Handling**: Implement robust error handling in your function call handlers to gracefully manage API failures or unexpected inputs.
2.  **Asynchronous Operations**: Explore how to handle long-running operations or asynchronous function calls within your assistant.
3.  **Complex Function Definitions**: Experiment with more complex function definitions, including nested objects and arrays in your parameters.
4.  **Integration with Real APIs**: Replace the mock functions with actual API calls to create truly interactive assistants.
5.  **Security Considerations**: If integrating with real APIs, be mindful of security best practices, such as API key management and input sanitization.

Happy building!